# HAMOYE STAGE C
<b>LESSON 1
    LOGISTIC REGRESSION</b>

<h3>Modified by <b><i>Miracle Osigwe</i></b></h3><br>
Hamoye ID <b>c9013e</b>

<h5>GENERAL NOTICE</h5>
<br>
The values on this notebook is bound to change, because random state is not set for the models<br>
used. So don't panic if you noticed the values does not tally with what you obtained.

In [1]:
#importing the necessary libraries
import pandas as pd
import numpy as np

# Please read this briefly

<p>I would presume you are running this on your local machine <br>
but if peradventure, you used Google colab to open this notebook,<br>
kindly input the url provided in cell 2 and pass it into the pandas read csv command in cell 4</p>

In [2]:
#using python to download the dataset (uncomment the codes below and execute them)
#import urllib.request
#url = 'https://query.data.world/s/wh6j7rxy2hvrn4ml75ci62apk5hgae'
#filename = 'dataworld.csv'
#urllib.request.urlretrieve(url, filename)

In [3]:
#using pandas to download the dataset (uncomment the code below and execute them)
#df = pd.read_csv('https://query.data.world/s/wh6j7rxy2hvrn4ml75ci62apk5hgae')
#df.to_csv('dataset.csv')

In [4]:
#load the dataset
df = pd.read_csv('dataset.csv', low_memory=False)
df.head()

,Unnamed: 0,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,0,Armenia,1992,1,AreaPerCap,0.140292,0.199546,0.097188051,0.036888,0.029320,0.000000,5.032351e-01,3A
1,1,Armenia,1992,1,AreaTotHA,483000.000000,687000.000000,334600,127000.000000,100943.000800,0.000000,1.732543e+06,3A
2,2,Armenia,1992,1,BiocapPerCap,0.159804,0.135261,0.084003213,0.013742,0.033398,0.000000,4.262086e-01,3A
3,3,Armenia,1992,1,BiocapTotGHA,550176.242700,465677.972200,289207.1078,47311.551720,114982.279300,0.000000,1.467355e+06,3A
4,4,Armenia,1992,1,EFConsPerCap,0.387510,0.189462,1.26E-06,0.004165,0.033398,1.114093,1.728629e+00,3A


In [5]:
#dropping the unnamed column (ensure you confirm it is there, else dont run this cell)

df.drop('Unnamed: 0', axis = 1, inplace = True)

In [6]:
#check the distribution of target variables
df['QScore'].value_counts()

3A    51481
2A    10576
2B    10096
1B       16
1A       16
Name: QScore, dtype: int64

# Note
The distribution of the target variables shows a huge differences between the classes

In [7]:
#checking for null values in the dataset
df.isnull().sum()
#or
df.isna().sum()

country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
QScore                1
dtype: int64

# Note 
From the dataset, 20,472 rows have missing values or null values present, which would affect our analysis

In [8]:
#for simplicity, we will drop the rows with missing values.
df.dropna(inplace = True)
df.isna().sum()

country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
QScore            0
dtype: int64

# Note
An obvious change in our target variable after removing the missing values is that there are only three classes left 
and from the distribution of the 3 classes, we can see that there is an obvious imbalance between the classes. 
There are methods that can be applied to handle this imbalance such as oversampling and undersampling.
Oversampling involves increasing the number of instances in the class with fewer instances, in this case the class with the lower instances while undersampling involves reducing the data points in the class with more instances in this case the 3A class.

In [9]:
#reset the dataframe index, to correct the effect of the null values dropped from the dataset
df = df.reset_index(drop = True)

In [10]:
#For now, we will convert this to a binary classification problem by combining class '2A' and '1A'.
df['QScore'] = df['QScore'].replace(['1A'], '2A')
df.QScore.value_counts()

3A    51473
2A      240
Name: QScore, dtype: int64

In [11]:
#separating the target variable and 
#undersampling the 3A class
df_2A = df[df.QScore=='2A']
df_3A = df[df.QScore=='3A'].sample(350)
data_df = df_2A.append(df_3A).reset_index(drop=True)
data_df

,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,Algeria,2016,4,AreaPerCap,2.072989e-01,8.112722e-01,0.048357265,2.258528e-02,2.998367e-02,0.000000e+00,1.119497e+00,2A
1,Algeria,2016,4,AreaTotHA,8.417600e+06,3.294260e+07,1963600,9.171000e+05,1.217520e+06,0.000000e+00,4.545842e+07,2A
2,Algeria,2016,4,BiocapPerCap,2.021916e-01,2.636077e-01,0.027166736,7.947991e-03,2.924496e-02,0.000000e+00,5.301590e-01,2A
3,Algeria,2016,4,BiocapTotGHA,8.210214e+06,1.070408e+07,1103135.245,3.227369e+05,1.187524e+06,0.000000e+00,2.152769e+07,2A
4,Algeria,2016,4,EFConsPerCap,6.280528e-01,1.810332e-01,0.162800822,1.472910e-02,2.924496e-02,1.391455e+00,2.407316e+00,2A
...,...,...,...,...,...,...,...,...,...,...,...,...
585,Argentina,2014,9,BiocapPerCap,2.577915e+00,1.792655e+00,0.611118727,1.612259e+00,1.045676e-01,0.000000e+00,6.698516e+00,3A
586,Albania,1984,3,AreaPerCap,2.721272e-01,2.189741e-01,0.395047858,2.203878e-01,1.624328e-02,0.000000e+00,1.122780e+00,3A
587,Saint Lucia,1977,189,EFProdTotGHA,3.475078e+04,1.412158e+03,3313.278393,7.791738e+03,2.939098e+02,2.490908e+04,7.247094e+04,3A
588,Mexico,1964,138,AreaTotHA,2.055000e+07,7.749900e+07,81629013.25,4.224940e+07,1.202400e+06,0.000000e+00,2.231298e+08,3A


In [12]:
#using the scikit-learn utils function to shuffle the target variable
import sklearn.utils
data_df = sklearn.utils.shuffle(data_df)
data_df = data_df.reset_index(drop=True)
data_df.shape
data_df.QScore.value_counts()

3A    350
2A    240
Name: QScore, dtype: int64

<h2>Note</h2>
A reset_index call and setting the drop to True helps to reset the index and make it reflect on the dataframe 

In [13]:
#check the datatype of the dataset
data_df.dtypes

country            object
year                int64
country_code        int64
record             object
crop_land         float64
grazing_land      float64
forest_land        object
fishing_ground    float64
built_up_land     float64
carbon            float64
total             float64
QScore             object
dtype: object

<h2>Note</h2>
From the above, we will be dropping country code, country and year, because they are ambiguous to what we want to predict<hr>
And also we will encode the record feature, <br>
Encoding is useful for columns that are needed but are not the target variable


In [14]:
#Encoding the categorical feature. 
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data_df.record = encoder.fit_transform(data_df.record)
#let's preview the encoded feature
data_df.record

0      5
1      0
2      6
3      3
4      0
      ..
585    2
586    4
587    6
588    0
589    7
Name: record, Length: 590, dtype: int32

In [15]:
#Preprocessing and uncorrelated columns dropping.
data_df = data_df.drop(columns=['country_code', 'country', 'year'])
X = data_df.drop(columns = 'QScore')
y = data_df['QScore']

In [16]:
#Splitting the data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
y_train.value_counts()

3A    231
2A    182
Name: QScore, dtype: int64

<h2>Note</h2>
The y train is showing a value counts of 245 to 168 for 3A and 2A respectively.<br>
which would overfit the model to make predictions towards 3A, because the model<br>
would have more features of 3A than 2A, and consequently make it probable to focus<br> more on 3A.<br>
To address this, we would be using imblearn to over sample our 2A class, so as to<br> create an equal feature weight

In [17]:
#installing imblearn module (uncomment the pip command to install imblearn), if already installed, ignore this cell
#!pip install imblearn

In [18]:
#There is still an imbalance in the class distribution. For this, we use SMOTE only on the training data to handle this.
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1)
x_train_balanced, y_balanced = smote.fit_sample(x_train, y_train)
#value count our y_balanced to cross check the effect of smote on the dataset
y_balanced.value_counts()

3A    231
2A    231
Name: QScore, dtype: int64

In [19]:
#using minmaxscaler to scale every other feature aside the record column
#because the record column is not a numerical feature but a categorical one

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

#scaling the x train set, without the record feature
normalised_train_df = scaler.fit_transform(x_train_balanced.drop(columns=['record']))

#passing the scaled data into a dataframe and setting the solumns without the record feature.
#the drop statement helps to ensure that the columns were not wrongly placed.
normalised_train_df = pd.DataFrame(normalised_train_df, columns=x_train_balanced.drop(columns=['record']).columns)

#adding the record columns back
normalised_train_df['record'] = x_train_balanced['record']

In [20]:
#Repeat the same for x test, 

#resetting the indexes for easy computatiion
x_test = x_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

normalised_test_df = scaler.transform(x_test.drop(columns=['record']))
normalised_test_df = pd.DataFrame(normalised_test_df, columns=x_test.drop(columns=['record']).columns)
normalised_test_df['record'] = x_test['record']

<h2>Note</h2>
<h4>The whole cell above were meant to prepare the data for the ML operations<br>
which would commence from the next cell below</h4>

In [21]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(normalised_train_df, y_balanced)
new_predictions = log_reg.predict(normalised_test_df)

In [22]:
#Runnning logisticregression.__dict__
LogisticRegression().__dict__
#returns
#LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
#                   intercept_scaling=1, l1_ratio=None, max_iter=100,
#                   multi_class='auto', n_jobs=None, penalty='l2',
#                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
#                   warm_start=False)

{'penalty': 'l2',
 'dual': False,
 'tol': 0.0001,
 'C': 1.0,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'class_weight': None,
 'random_state': None,
 'solver': 'lbfgs',
 'max_iter': 100,
 'multi_class': 'auto',
 'verbose': 0,
 'warm_start': False,
 'n_jobs': None,
 'l1_ratio': None}

<h1>LESSON 2</h1>
<b>MEASURING CLASSIFICATION PERFORMANCE</b>

In [23]:
#cross validation score
from sklearn.model_selection import cross_val_score
scores = cross_val_score(log_reg, normalised_train_df, y_balanced, cv=5,
                        scoring='f1_macro')
scores*100

array([54.41176471, 50.53191489, 48.61556744, 52.92397661, 58.38095238])

In [24]:
#LeaveOneOut
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(LogisticRegression(), normalised_train_df, y_balanced, \
                        cv=loo, scoring='f1_macro')
average_score = scores.mean() * 100
print("Average score %.2f" % average_score)

Average score 54.33


In [25]:
#confusion matrix
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
cnf_mat = confusion_matrix(y_true=y_test, y_pred=new_predictions, labels=['2A', '3A'])

#Preview the confusion matrix
good_cnf = pd.DataFrame(cnf_mat, columns=['Actual- Positive', 'Actual- Negative'], 
                        index=['Predicted- Positive', 'Predicted- Negative'])
good_cnf

,Actual- Positive,Actual- Negative
Predicted- Positive,31,27
Predicted- Negative,65,54


In [26]:
print("%.0f is the number of True Positive (TP)" % (cnf_mat[0, 0]))
print("%.0f is the number of False Positive (FP)" % (cnf_mat[0, 1]))
print("%.0f is the number of False Negative (FN)" % (cnf_mat[1, 0]))
print("%.0f is the number of True Negative (TN)" % (cnf_mat[1, 1]))

31 is the number of True Positive (TP)
27 is the number of False Positive (FP)
65 is the number of False Negative (FN)
54 is the number of True Negative (TN)


<h2>METRICS</h2> 

In [27]:
#Accuracy
accuracy = accuracy_score(y_true = y_test, y_pred = new_predictions)
print('Accuracy: {}'.format(round(accuracy*100, 2)))

Accuracy: 48.02


In [28]:
#precision
precision = precision_score(y_true=y_test, y_pred=new_predictions, \
                           pos_label='2A')
print('Presicion: {}'.format(round(precision*100), 2))

Presicion: 32.0


In [29]:
#recall
recall = recall_score(y_true=y_test, y_pred=new_predictions, \
                           pos_label='2A')
print('Recall: {}'.format(round(recall*100), 2))

Recall: 53.0


In [30]:
#F1 scores
f1 = f1_score(y_true=y_test, y_pred=new_predictions, pos_label='2A')
print('F1: {}'.format(round(f1*100), 2))

F1: 40.0


<h3> K-FOLD </h3>
<i>Kindly read through and let me know if you need further assistance with any section of the code</i>

In [31]:
#K-Fold
#calculating the classification report for kfold

from sklearn.model_selection import KFold

#setting the number of splits
kf = KFold(n_splits=5)
kf.split(normalised_train_df)
#metric lists to hold the values calculated for every k-fold carried out
f1_scores, accuracy_kfold, precision_kfold, recall_kfold = [], [], [], []
kmodel = []

#run for every split
for train_index, test_index in kf.split(normalised_train_df):
    kx_train, kx_test = normalised_train_df.iloc[train_index],\
                      normalised_train_df.iloc[test_index]
    ky_train, ky_test = y_balanced[train_index],\
                      y_balanced[test_index]
    model = LogisticRegression().fit(kx_train, ky_train)
    #save result to list
    f1_scores.append(f1_score(y_true = ky_test, y_pred = model.predict(kx_test),
                            pos_label = '2A'))
    accuracy_kfold.append(accuracy_score(y_true=ky_test, y_pred=model.predict(kx_test)))
    precision_kfold.append(precision_score(y_true = ky_test, y_pred = model.predict(kx_test), 
                                           pos_label = '2A', zero_division=True))
    recall_kfold.append(recall_score(y_true = ky_test, y_pred = model.predict(kx_test),
                            pos_label = '2A'))
    pred = model.predict(kx_test)
    kmodel.append(pred)

In [32]:
#Passing the reports into a dataframe
classification_scores = pd.DataFrame([accuracy_kfold, precision_kfold, recall_kfold, f1_scores], index=['Accuracy', 
                                                                                                          'Precision',
                                                                                                          'Recall',
                                                                                                         'F1 Scores'],
                                    columns=['k1', 'k2', 'k3', 'k4', 'k5'])
classification_scores

,k1,k2,k3,k4,k5
Accuracy,0.559140,0.516129,0.500000,0.500000,0.271739
Precision,0.537037,0.510638,0.426230,0.421875,1.000000
Recall,0.644444,0.521739,0.702703,0.750000,0.000000
F1 Scores,0.585859,0.516129,0.530612,0.540000,0.000000


<h2>Note</h2>
<h3>CLASSIFICATION REPORT</h3>
Classification report is useful for models that uses classifiers for it's algorithms.<br>
<p></p>
Try comparing the metrics given by the classification report and the ones given by each<br>
metrics, you will notice a difference in value.<br>
Should incase you have an explanation to this, kindly use the comment section to help clarify this<br>
Also all the metrics used from this point downward are in decimal places and not percentage, for easy comparison

In [33]:
from sklearn.metrics import classification_report

In [34]:
#classification report
if len(ky_test) != len(kmodel[0]):
    kmodels = kmodel[0][:-1]
    print(classification_report(ky_test, kmodels))
else:
    print(classification_report(ky_test, kmodel[0]))

              precision    recall  f1-score   support

          2A       0.79      0.63      0.70        67
          3A       0.36      0.56      0.44        25

    accuracy                           0.61        92
   macro avg       0.58      0.59      0.57        92
weighted avg       0.67      0.61      0.63        92



In [35]:
#classification report
if len(ky_test) != len(kmodel[1]):
    kmodels = kmodel[1][1:]
    print(classification_report(ky_test, kmodels))
else:
    print(classification_report(ky_test, kmodel[1]))

              precision    recall  f1-score   support

          2A       0.72      0.49      0.58        67
          3A       0.26      0.48      0.34        25

    accuracy                           0.49        92
   macro avg       0.49      0.49      0.46        92
weighted avg       0.59      0.49      0.52        92



In [36]:
#You can experiment for other splits by using the slice (2, 3, 4) inside the previous classification reports

<h3>Stratified K-FOLD</h3>

In [37]:
#StratifiedKFold
#Calculating the classification reports skfold 

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
f1_scores_skfold, accuracy_skfold, precision_skfold, recall_skfold, cnf_mtxS = [], [], [], [], []
skmodel=[]

#run for every split
for train_index, test_index in skf.split(normalised_train_df, y_balanced):
    skx_train, skx_test = np.array(normalised_train_df)[train_index],\
                      np.array(normalised_train_df)[test_index]
    sky_train, sky_test = y_balanced[train_index],\
                      y_balanced[test_index]
    model = LogisticRegression().fit(skx_train, sky_train)
    #save result to list
    f1_scores_skfold.append(f1_score(y_true = sky_test, y_pred = model.predict(skx_test),
                            pos_label = '2A'))
    accuracy_skfold.append(accuracy_score(y_true=sky_test, y_pred=model.predict(skx_test)))
    precision_skfold.append(precision_score(y_true = sky_test, y_pred = model.predict(skx_test), 
                                            pos_label = '2A', zero_division=True))
    recall_skfold.append(recall_score(y_true = sky_test, y_pred = model.predict(skx_test),
                            pos_label = '2A'))
    skmodel.append(model.predict(skx_test))

In [38]:
#Passing the values into a dataframe
classification_scores2 = pd.DataFrame([accuracy_skfold, precision_skfold, recall_skfold, f1_scores_skfold], index=['Accuracy', 
                                                                                                          'Precision',
                                                                                                          'Recall',
                                                                                                         'F1 Scores'],
                                    columns=['sk1', 'sk2', 'sk3', 'sk4', 'sk5'])
classification_scores2

,sk1,sk2,sk3,sk4,sk5
Accuracy,0.537634,0.591398,0.641304,0.423913,0.510870
Precision,0.533333,0.576923,0.638298,0.410256,0.508772
Recall,0.680851,0.652174,0.652174,0.347826,0.630435
F1 Scores,0.598131,0.612245,0.645161,0.376471,0.563107


In [39]:
#classification report
if len(sky_test) != len(skmodel[0]):
    skmodels = skmodel[0][:-1]
    print(classification_report(sky_test, skmodels))
else:
    print(classification_report(sky_test, skmodel[0]))

              precision    recall  f1-score   support

          2A       0.47      0.61      0.53        46
          3A       0.44      0.30      0.36        46

    accuracy                           0.46        92
   macro avg       0.45      0.46      0.44        92
weighted avg       0.45      0.46      0.44        92



In [40]:
#classification report
if len(sky_test) != len(skmodel[1]):
    skmodels = skmodel[1][1:]
    print(classification_report(sky_test, skmodels))
else:
    print(classification_report(sky_test, skmodel[1]))

              precision    recall  f1-score   support

          2A       0.46      0.52      0.49        46
          3A       0.45      0.39      0.42        46

    accuracy                           0.46        92
   macro avg       0.46      0.46      0.45        92
weighted avg       0.46      0.46      0.45        92



In [41]:
#You can experiment for other splits by using the slice (2, 3, 4) inside the previous classification reports

<h3>DECISION TREE CLASSIFIER</h3>

In [42]:
# Tree-Based Methods and The Support Vector Machine
from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier()
dec_tree.fit(normalised_train_df, y_balanced)
dec_pred = dec_tree.predict(normalised_test_df)

In [43]:
print("Accuracy score {}".format(round(accuracy_score(y_test, dec_pred), 3)))
print("Precision score for label 2A %.3f" % (precision_score(y_test, dec_pred, pos_label='2A')))
print("Recall score for label 2A {}".format(round(recall_score(y_test, dec_pred, pos_label='2A'), 3)))
print("F1 score %.3f" % (f1_score(y_test, dec_pred, pos_label='2A')))

Accuracy score 0.599
Precision score for label 2A 0.393
Recall score for label 2A 0.414
F1 score 0.403


In [44]:
print(classification_report(y_test, dec_pred))

              precision    recall  f1-score   support

          2A       0.39      0.41      0.40        58
          3A       0.71      0.69      0.70       119

    accuracy                           0.60       177
   macro avg       0.55      0.55      0.55       177
weighted avg       0.60      0.60      0.60       177



<h2>WORKING WITH XGBOOST AND LIGHTGBM</h2>

In [45]:
#testing two xgboost classifier

<h2>XGBRFCLASSIFIER</h2>
XGBoost is normally used to train gradient-boosted decision trees<br> 
and other gradient boosted models. Random forests use the same model<br> 
representation and inference, as gradient-boosted decision trees,<br> 
but a different training algorithm. One can use XGBoost to train a<br> 
standalone random forest or use random forest as a base model for<br> 
gradient boosting. Here we focus on training standalone random forest.

<h2>XGBCLASSIFIER</h2>
XGBoost is a decision-tree-based ensemble Machine Learning algorithm<br> 
that uses a gradient boosting framework. In prediction problems involving<br> 
unstructured data (images, text, etc.) artificial neural networks tend to<br> 
outperform all other algorithms or frameworks. However, when it comes to<br> 
small-to-medium structured/tabular data, decision tree based algorithms are<br> 
considered best-in-class right now.

In [46]:
#xgboost with random forest
from xgboost import XGBRFClassifier
extreme = XGBRFClassifier()
extreme.fit(normalised_train_df, y_balanced)
extreme_pred = extreme.predict(normalised_test_df)

In [47]:
#classification report
print(classification_report(y_test, extreme_pred))

              precision    recall  f1-score   support

          2A       0.50      0.66      0.57        58
          3A       0.80      0.68      0.74       119

    accuracy                           0.67       177
   macro avg       0.65      0.67      0.65       177
weighted avg       0.70      0.67      0.68       177



In [48]:
#xgboost using gradient boosting
from xgboost import XGBClassifier
extreme1 = XGBClassifier()
extreme1.fit(normalised_train_df, y_balanced)
extreme1_pred = extreme1.predict(normalised_test_df)

In [49]:
#classification report
print(classification_report(y_test, extreme1_pred))

              precision    recall  f1-score   support

          2A       0.51      0.62      0.56        58
          3A       0.79      0.71      0.75       119

    accuracy                           0.68       177
   macro avg       0.65      0.67      0.66       177
weighted avg       0.70      0.68      0.69       177



In [50]:
#lightgbm
from lightgbm import LGBMClassifier
light = LGBMClassifier()
light.fit(normalised_train_df, y_balanced)
light_pred = light.predict(normalised_test_df)

In [51]:
#classification report
print(classification_report(y_test, light_pred))

              precision    recall  f1-score   support

          2A       0.49      0.59      0.53        58
          3A       0.78      0.70      0.73       119

    accuracy                           0.66       177
   macro avg       0.63      0.64      0.63       177
weighted avg       0.68      0.66      0.67       177



<h2>EXTRA TREE CLASSIFIER</h2>

In [52]:
#extra tree classifier
from sklearn.ensemble import ExtraTreesClassifier
tree = ExtraTreesClassifier()
tree.fit(normalised_train_df, y_balanced)
tree_pred = tree.predict(normalised_test_df)

<h3>Tree tuning using hyper parameters setting</h3>

Now to improve the Extra Trees Classifier, you will use the following parameters<br> 
<ul>
    <ol>Number of estimators</ol> 
    <ol>Minimum number of samples,</ol> 
    <ol>Minimum number of samples for leaf node and </ol>
    <ol>The number of features to consider when looking for the best split</ol>
</ul>    
For the hyperparameter grid needed to run a Randomized Cross Validation Search (RandomizedSearchCV). 


In [53]:
n_estimators = [10, 50, 100, 250, 400]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None] 
hyperparameter_grid = {'n_estimators': n_estimators,
                       'min_samples_leaf': min_samples_leaf,
                       'min_samples_split': min_samples_split,
                       'max_features': max_features}

In [54]:
from sklearn.model_selection import RandomizedSearchCV
clf = RandomizedSearchCV(tree, hyperparameter_grid, random_state=0)
search = clf.fit(normalised_train_df, y_balanced)

In [55]:
#checking for the best parameter for the model
search.best_params_.values()

dict_values([400, 2, 1, 'log2'])

In [56]:
#experimenting with this parameter to test the model's performance
tree_param = ExtraTreesClassifier(n_estimators=400, min_samples_split=2, 
                                 min_samples_leaf=1, max_features='log2')
tree_param.fit(normalised_train_df, y_balanced)
tree_param_pred = tree_param.predict(normalised_test_df)

In [57]:
#classification report for this hyperparameter tuning
print(classification_report(y_test, tree_param_pred))

              precision    recall  f1-score   support

          2A       0.29      0.28      0.28        58
          3A       0.65      0.66      0.66       119

    accuracy                           0.54       177
   macro avg       0.47      0.47      0.47       177
weighted avg       0.53      0.54      0.53       177



<h2>Classification Report clarification</h2>

Please use the comment section to help clarify these reports and their significances.<br>
Thanks.

<h1>NOTE</h1>
<h3>IF THIS NOTEBOOK WAS HELPFUL, KINDLY VOTE FOR ME USING THESE DETAILS</h3>
<h2> Name: OSIGWE MIRACLE</h2>
<h3>ID: c9013e</h3>